In [ ]:
import os
str1 = ''
file_list = []

os.chdir(r"F:\河南职务侵占清洗")
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
            
print(len(file_list))
print('\n'.join(file_list))


import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
type3 = '.et'
output_path = 'C:\\Users\\ThinkPad\\Desktop\\河南\\合并\\'

for i in range(len(file_list)):
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], encoding = 'GB18030')
        df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    else:
        continue
    print(i ,end = ' ')
    df.columns = df.columns.str.strip()
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

import pandas as pd
import numpy as np
import re
import datetime

all_流水 = pd.DataFrame()

for i in range(len(file_list)):
    Efile =  pd.ExcelFile(file_list[i])
    sheet_list = pd.ExcelFile(file_list[i]).sheet_names
    for k in range(0,len(sheet_list)):
        if sheet_list[k] != '':
            df = Efile.parse(sheet_name = sheet_list[k],dtype = 'object')
            df['path'] = file_list[i]
            all_流水 = all_流水.append(df)
            print(str(i)+'-'+str(k),end = ' ')
        else:
            continue
all_流水.to_excel(output_path + '合并.xlsx',index = None)
print('Done!')

In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import os

output_path = r'C:\Users\ThinkPad\Desktop\河南第二批'
output_path = eval(repr(output_path))+str('\\')

df = pd.read_excel(r"C:\Users\ThinkPad\Desktop\河南第二批\合并\银行交易流水.xlsx",encoding = "gbk",dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))

df['序号'] = df.index
# 24小时标记
df['计时标记'] = ''
all_流水完整 = df

def 计时12to24标记(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['交易时间'].iloc[k][11:19] < df_计时['交易时间'].iloc[(k1)][11:19]):
            df_计时['计时标记'].iloc[k] = '+12'
    return df_计时

def 计时24之后标记(df_计时):
    for k in range(1,len(df_计时)):
        k1 = k-1
        if df_计时['交易时间'].iloc[k] is None:
            continue
        elif (df_计时['借贷标志'].iloc[k] == '出') and (df_计时['查询卡号'].iloc[k] == df_计时['查询卡号'].iloc[(k1)]) and (df_计时['交易时间'].iloc[k][0:10] == df_计时['交易时间'].iloc[(k1)][0:10]) and (df_计时['计时标记'].iloc[(k1)]=='+12'):
            df_计时['计时标记'].iloc[k] = '+12'
    return df_计时


def index向后N位的index(df_序列, j_一方, no_N):
    for k in df_序列.index:
        if k >j_一方:
            no_序列号 = len(df_序列.loc[:k] )
            no_目标序列号 = no_序列号 + no_N
            if no_目标序列号 >= len(df_序列):
                index_目标 = df_序列.index.max()
            else:
                index_目标 = df_序列.reset_index().iloc[ no_目标序列号-1 ]['index']
            break
    return index_目标

def 一方匹配另一方(df_一方, all_另一方,tablename_索引,sheetname_层数):
    df_一方.index = df_一方['序号'].to_list()
    df_一方 = df_一方[df_一方['借贷标志']=='进'].drop_duplicates(subset = ['序号'],keep = 'last')
    df_匹配 = pd.DataFrame()
    df_未匹配 = pd.DataFrame()
    df_另一方 = pd.DataFrame(all_另一方[ (all_另一方['借贷标志']=='出')  ])
    print(str(len(df_一方))+"条，最大索引为"+ str(df_一方.index.max()))

    for j in df_一方.index:
        for N in range(100):
            x = index向后N位的index(df_另一方, j, N)
    
            df_一进一出 = df_另一方[( df_另一方['序号'] <= x ) & ( df_另一方['序号'] > j ) \
                & (df_另一方['查询卡号']== df_一方['查询卡号'].loc[j] ) \
                ]
            if df_一方['交易金额'].loc[j]*(1-0.10) < df_一进一出['交易金额'].sum(): ############下浮10%###########
                break
            else:
                continue
        if len(df_一进一出)>0:

            
            merge_一进一出 = pd.merge(df_一方.loc[[j],:],df_一进一出,how = 'outer')
            df_匹配 = df_匹配.append(merge_一进一出)
            # 24小时标记
            df_匹配 = 计时12to24标记(df_匹配)
            df_匹配 = 计时24之后标记(df_匹配)
            # 24小时标记
            print(str(j)+"-"+str(x), end = ' ')
        elif len(df_一进一出)==0:
            df_未匹配 = df_未匹配.append(df_一方.loc[[j],:])
            print(str(j)+'失败', end = ' ')

    if not os.path.isfile(output_path + tablename_索引+ "追踪转移.xlsx"):
        df_匹配.to_excel(output_path + tablename_索引+ "追踪转移.xlsx", sheet_name= sheetname_层数,index = False)
    else:
        with pd.ExcelWriter(output_path + tablename_索引+ "追踪转移.xlsx" ,mode='a', engine='openpyxl') as writer:
            df_匹配.to_excel(writer, sheet_name= sheetname_层数,index = False)
    return df_匹配

def AB匹配(df_A,df_B,tablename_索引,sheetname_层数):
    df_A.index = df_A['序号'].to_list()
    df_A出B进 = pd.DataFrame()
    df_A出B空 = pd.DataFrame()
    df_A出 = df_A[df_A['借贷标志']=='出'].drop_duplicates(subset = ['序号'],keep = 'last')
    for k in df_A出.index:
        df_B进 = df_B[(df_B['借贷标志']=='进') \
            & ( df_B['交易对方卡号'].isin(df_A出['查询卡号']) ) \
            & (df_B['交易时间']<= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes = 1) )  ) \
            & (df_B['交易时间']>= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds = -1) ) ) \
            & (df_B['交易金额'] == df_A出['交易金额'].loc[k])  \
            ]
        if len(df_B进)>0:
            # 24小时标记
            df_B进['计时标记'] = df_A出['计时标记'].loc[k]
            # 24小时标记
            merge_A出B进 = pd.merge(df_A出.loc[[k],:], df_B进, how = 'outer')
            df_A出B进 = df_A出B进.append(merge_A出B进)
            print(k, end = ' ')
        elif len(df_B进)==0:
            df_B进 = df_B[(df_B['借贷标志']=='进') \
            & ( df_B['交易对方卡号'].isin(df_A出['查询卡号']) ) \
            & (df_B['交易时间']<= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S')+datetime.timedelta(hours = 12) +datetime.timedelta(minutes = 1) )  ) \
            & (df_B['交易时间']>= str(datetime.datetime.strptime(df_A出['交易时间'].loc[k], '%Y-%m-%d %H:%M:%S')+datetime.timedelta(hours = 12) + datetime.timedelta(seconds = -1) ) ) \
            & (df_B['交易金额'] == df_A出['交易金额'].loc[k])  \
            ] 
            if len(df_B进)>0:
                merge_A出B进 = pd.merge(df_A出.loc[[k],:], df_B进, how = 'outer')
                df_A出B进 = df_A出B进.append(merge_A出B进)
                print(k, end = ' ')
            elif len(df_B进)==0:
                df_A出B空 = df_A出B空.append(df_A出.loc[[k],:])
                print(str(k)+'失败', end = ' ')
#     if not os.path.isfile(output_path + tablename_索引+ "转移未调.xlsx"):
#         df_A出B空.to_excel(output_path + tablename_索引+ "转移未调.xlsx", sheet_name= sheetname_层数,index = False)
#     else:
#         with pd.ExcelWriter(output_path + tablename_索引+ "转移未调.xlsx" ,mode='a', engine='openpyxl') as writer:
#             df_A出B空.to_excel(writer, sheet_name= sheetname_层数,index = False)
#     if not os.path.isfile(output_path + tablename_索引+ "转移已调.xlsx"):
#         df_A出B进.to_excel(output_path + tablename_索引+ "转移已调.xlsx", sheet_name= sheetname_层数,index = False)
#     else:
#         with pd.ExcelWriter(output_path + tablename_索引+ "转移已调.xlsx" ,mode='a', engine='openpyxl') as writer:
#             df_A出B进.to_excel(writer, sheet_name= sheetname_层数,index = False)
    return df_A出B进

def digui多层(df_层出层进,tablename_索引):
    if len(df_层出层进)>0:
        df_某层 = 一方匹配另一方(df_层出层进,all_流水完整,tablename_索引,'层')
        df_某层A出B进 = AB匹配(df_某层,all_流水完整,tablename_索引,'层')
        digui多层(df_某层A出B进,tablename_索引)
        
def 合并多sheet24(多sheet_filename,tablename_索引):
    all_流水 = pd.DataFrame()
    Efile =  pd.ExcelFile(多sheet_filename)
    sheet_list = pd.ExcelFile(多sheet_filename).sheet_names
    for j in range(0,len(sheet_list)):
        if sheet_list[j] != '':
            df = Efile.parse(sheet_name = sheet_list[j],dtype = 'object')
            df['交易时间'] = df.apply(lambda x: str(datetime.datetime.strptime(x['交易时间'], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(hours = 12) ) if x['计时标记']=='+12' else x['交易时间'],axis=1)
            all_流水 = all_流水.append(df)
            
            print(str('')+'-'+str(j),end = ' ')
        else:
            continue
    all_流水.to_excel(output_path + tablename_索引+ "追踪转移合并.xlsx",index = False)

In [4]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=10000000 )
#                  & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')]
for i in df_原点线索.index:
    t_索引 = str(i)
    digui多层(df_原点线索.loc[[i],:],t_索引)
    file_name_t = output_path + t_索引+ "追踪转移.xlsx"
    合并多sheet24(file_name_t,t_索引)

1条，最大索引为102293
102293-102302 102295失败 102296失败 102297失败 102299 102300 102302 3条，最大索引为103361


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


69840-69842 66365-66366 103361-103364 69841失败 69842失败 66366失败 103364 1条，最大索引为102908
102908-102931 102909失败 102910失败 102911失败 102912失败 102913失败 102914失败 102915失败 102916失败 102917失败 102918失败 102931失败 -0 -1 -2 1条，最大索引为102298
102298-102302 102299 102300 102302 3条，最大索引为103361
69840-69842 66365-66366 103361-103364 69841失败 69842失败 66366失败 103364 1条，最大索引为102908
102908-102931 102909失败 102910失败 102911失败 102912失败 102913失败 102914失败 102915失败 102916失败 102917失败 102918失败 102931失败 -0 -1 -2 1条，最大索引为102310
102310-102312 102312 1条，最大索引为69852
69852-69853 69853失败 -0 -1 1条，最大索引为102317
102317-102319 102318 102319 2条，最大索引为69863
69859-69860 69863-69865 69860失败 69864失败 69865失败 -0 -1 1条，最大索引为102326
102326-102328 102327失败 102328失败 -0 1条，最大索引为102330
102330-102332 102332失败 -0 1条，最大索引为102336
102336-102337 102337失败 -0 1条，最大索引为102338
102338-102340 102339 102340失败 1条，最大索引为69903
69903-69908 69904 69907失败 69908失败 1条，最大索引为66387
66387-66391 66388 66391失败 1条，最大索引为69906
69906-69908 69907失败 69908失败 -0 -1 -2 -3 1条，最大索引为102341
10

102597-102598 70071-70072 102600-102601 102598失败 70072 102601失败 1条，最大索引为102599
102599-102601 102601失败 -0 -1 -2 1条，最大索引为103468
103468-103471 103469失败 103470失败 103471失败 -0 1条，最大索引为103485
103485-103486 103486 1条，最大索引为106744
106744-106745 106745 1条，最大索引为85019
85019-85020 85020 1条，最大索引为103487
103487-103488 103488 1条，最大索引为106746
106746-106747 106747 1条，最大索引为85021
85021-85022 85022 1条，最大索引为103489
103489-103490 103490 1条，最大索引为106748
106748-106754 106751失败 106753失败 106754失败 -0 -1 -2 -3 -4 -5 -6 -7 1条，最大索引为103487
103487-103488 103488 1条，最大索引为106746
106746-106747 106747 1条，最大索引为85021
85021-85022 85022 1条，最大索引为103489
103489-103490 103490 1条，最大索引为106748
106748-106754 106751失败 106753失败 106754失败 -0 -1 -2 -3 -4 1条，最大索引为103489
103489-103490 103490 1条，最大索引为106748
106748-106754 106751失败 106753失败 106754失败 -0 -1 1条，最大索引为103498
103498-103499 103499失败 -0 1条，最大索引为103563
103563-103565 103564失败 103565 1条，最大索引为107731
107731-107747 107732失败 107734 107741失败 107742失败 107743失败 107744失败 107745失败 107746失败 107747失败 1条，

104202-104203 104203 1条，最大索引为88671
88671-88672 88672失败 -0 -1 1条，最大索引为104204
104204-104205 104205 1条，最大索引为88673
88673-88675 88675 1条，最大索引为105600
105600-105601 105601失败 -0 -1 -2 1条，最大索引为104207
104207-104209 104208 104209 2条，最大索引为88677
88676-88679 88677-88679 88679 1条，最大索引为105602
105602-105603 105603失败 -0 -1 -2 1条，最大索引为104210
104210-104211 104211 1条，最大索引为88685
88685-88686 88686失败 -0 -1 1条，最大索引为104221
104221-104222 104222 1条，最大索引为93743
93743-93744 93744失败 -0 -1 1条，最大索引为104245
104245-104250 104246 104247 104248 104249 104250 5条，最大索引为89616
89612-89617 89613-89617 89614-89617 89615-89617 89616-89617 89617失败 -0 -1 1条，最大索引为104260
104260-104274 104261 104262 104263 104264 104265 104266 104267 104268 104269 104270 104271 104272 104273 104274 14条，最大索引为89751
89738-89753 89739-89753 89740-89753 89741-89753 89742-89753 89743-89753 89744-89753 89745-89753 89746-89753 89747-89753 89748-89753 89749-89753 89750-89753 89751-89753 89753失败 -0 -1 1条，最大索引为104289
104289-104296 104296失败 -0 1条，最大索引为104290
104290

In [3]:
all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=50000000 )
#                  & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')].to_excel(output_path +"单笔索引.xlsx",sheet_name = '大于等于五千万',index = None)

with pd.ExcelWriter(output_path + "单笔索引.xlsx" ,mode='a', engine='openpyxl') as writer:
    all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=30000000 )
                 & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')].to_excel(writer,sheet_name = '大于等于三千万小于五千万',index = None)
    all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=10000000 )
                 & (all_流水完整['交易金额']<30000000 )
                 & (all_流水完整['借贷标志']=='进')].to_excel(writer,sheet_name = '大于等于一千万小于三千万',index = None)

In [20]:


all_流水完整[(all_流水完整['交易对方卡号']=='8981214000000002034')
        & (all_流水完整['查询卡号']=='79170155200006355') 
        & (all_流水完整['交易时间'] >= '2015-09-25 14:41:00') ].iloc[:,:16]

,银行,客户名称,客户证件号码,查询账号,查询卡号,交易对方名称,交易对方证件号码,交易对方账号,交易对方卡号,交易类型,借贷标志,币种,交易金额,交易余额,交易时间,交易流水号
52338,,深圳宜投基金销售有限公司,,,79170155200006355,河南蔡氏餐饮管理有限公司,,,8981214000000002034,,进,,129500.0,2101277.75,2015-09-25 14:41:00,
52339,,深圳宜投基金销售有限公司,,,79170155200006355,河南蔡氏餐饮管理有限公司,,,8981214000000002034,,进,,70000.0,2171277.75,2015-09-25 14:47:44,


In [3]:
all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=50000000 )
#                  & (all_流水完整['交易金额']<50000000 )
                 & (all_流水完整['借贷标志']=='进')].iloc[:,:16]

,银行,客户名称,客户证件号码,查询账号,查询卡号,交易对方名称,交易对方证件号码,交易对方账号,交易对方卡号,交易类型,借贷标志,币种,交易金额,交易余额,交易时间,交易流水号
102377,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,先锋创业有限公司,,,44201017100052509986,,进,人民币,5.000000e+07,5.006446e+07,2014-11-10 04:44:20,
102404,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,河南迈乐加餐厅食品有限公司,,,8981214000000004997,,进,人民币,6.805879e+07,6.865423e+07,2014-12-24 11:42:48,
102411,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,大连润和贸易有限公司,,,35710188000215331,,进,人民币,7.500000e+07,1.215954e+08,2015-01-06 10:16:27,
102414,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,河南迈乐加餐厅食品有限公司,,,602180177210001,,进,人民币,1.216696e+08,1.216954e+08,2015-01-06 03:22:47,
102503,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,,,,,,进,人民币,6.976000e+07,8.706148e+07,2015-08-06 05:06:48,
102513,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,,,,,,进,人民币,5.929000e+07,6.288214e+07,2015-08-21 04:06:49,
102594,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,河南蔡氏餐饮管理有限公司,,,8981215030000000462,,进,人民币,5.502049e+07,5.559461e+07,2015-10-19 02:58:54,
102597,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,河南迈乐加餐厅食品有限公司,,,8981214000000004997,,进,人民币,5.000000e+07,5.506961e+07,2015-10-27 02:04:58,
102620,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,河南新家园建材家居有限公司,,,00000216024920070012,,进,人民币,5.000000e+07,5.009398e+07,2015-12-09 03:48:31,
102638,,河南蔡氏餐饮管理有限公司,,,8981214000000002034,,,,,,进,人民币,7.047000e+07,8.520508e+07,2016-01-18 05:01:51,


In [5]:
df_原点线索 = all_流水完整[ (all_流水完整['客户名称'].isin(['河南迈乐加餐厅食品有限公司','河南蔡氏餐饮管理有限公司']) )
                 & (all_流水完整['交易金额']>=3580000 )
                 & (all_流水完整['交易金额']<3590000 )
                 & (all_流水完整['借贷标志']=='进')]
for i in df_原点线索.index:
    t_索引 = str(i)
    digui多层(df_原点线索.loc[[i],:],t_索引)
    file_name_t = output_path + t_索引+ "追踪转移.xlsx"
    合并多sheet24(file_name_t,t_索引)

1条，最大索引为103977
103977-103978 103978失败 -0 1条，最大索引为103980
103980-103983 

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


103982 103983 2条，最大索引为93625
86941-86943 93625-93631 86942失败 86943失败 93626 93627 93631失败 2条，最大索引为103985
103984-103986 103985-103986 103986 1条，最大索引为86959
86959-86967 86967 1条，最大索引为103988
103988-103989 103989 1条，最大索引为93628
93628-93631 93631失败 -0 -1 -2 -3 -4 -5 